In [ ]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

In [ ]:
using DelimitedFiles # read and write plain txt files
using TypedTables # form tables
using Plots # plotting 
using Optim # fitting
using Parameters # @unpack, working with the named tupples

In [ ]:
theme(:wong)

# Resonance analysis

1. Intro: complex functions
2. A pole of $a_2(1320)$
3. Advancing the model: good fit
4. Extending the range: the pole of $a_2'(1700)$

## 1.0 Intro: Scattering amplitude
The observed intensity (cross section / differential width) of the QM process:
$$
\begin{align}
I(s) = \frac{1}{J} \int |M|^2 d\Phi
\end{align}
$$
where
$$
M = \left\langle \text{assymp. final state}| \hat{T} | \text{assymp. initial state}\right\rangle
$$
(there is also delta function of the what we will often omit):
$$
(2\pi)^4\delta^4(\text{energy-mom. conservation})
$$

$M$ is the matrix element, or **scattering amplitude** - a dynamic function containing all information about the scattering.
It can also be called
 * transition amplitude
 * production amplitude: the particle species in the initial state are different from the final one

Two things to distinguish:
 - **Elastic** process: $\pi\pi\to\pi\pi$ in $P$-wave below $1\,$GeV - no other possible outcomes of the initial states
 - **Inelastic** process: $\pi\pi\to K\bar{K}$ in $S$-wave above $1\,$GeV - there are other possibilities for the final state when you bring two particles together.
 
 ***Wave***??? - *the quantum numbers of the system) - the total angural momenum $J$. Concerved!*

#### The phase space
$\Phi$ is the number of the possible configrations of the system.

Two particle system of $m_1 = 0$, $m_2 = 0$, with the energy $E = 5\,$GeV
 - $E_1 = 2.5\,$GeV, $E_2=2.5\,$GeV
 - the phase space is equal to the possible configutations are their angle

$$
\Phi = n_\Phi d\Omega = n_\Phi (4\pi)
$$
the full solid angle, ($n_\Phi = 1/(8\pi)/(4\pi)$ - the normalization).

In [ ]:
Φ = 1/(8π)
M = 3+1im

In [ ]:
Intensity = abs2(M)*Φ # not the probability yet - more like cross section (the flux is missing)

What the $M$ depends on? 
  - most importantly, on the **energy of the system**. The energy is conserved between the initial and the final state.
$$
s = (p_1+p_2)^2,\qquad\text{still the energy of the system, but in the Lorentz invariant form}
$$
Analog of the frequency: the system might start resonating if we ajust the "frequency" well to fit the mass of the other particle.

Mathematically, $M(s)$ is a complex function. The poles of $M$ are the resonances: they correspond to other particles.
$$
\begin{align}
    M(s) \xrightarrow[s\to s_\text{pole}^R]{} \frac{\text{res}}{s_\text{pole}^R-s},
    \qquad
    s_\text{pole}^R = (m_R - \frac{i\Gamma}{2})^2.
\end{align}
$$
$s_\text{pole}^R$ is universal propery of the resonance: does not matter which system resonates - the pole is fixed for $R$.

In [ ]:
Mres(s) = 1/(s^2+3^2)

In [ ]:
plot(Mres, -30, 30, lab="my first amplitude")

In [ ]:
let
    xv = range(-10,10, length=100)
    yv = range(-10,10, length=100)
    calv = [Mres(x+1im*y) for y in yv, x in xv]
    plot3d(xv,yv,log.(abs2.(calv)))
end

***Exercise:*** Come up with an amplitude that
 - have a pole at the at $(m-i\Gamma/2) = (1.3-0.01i)$
 - have two poles below the real axis

## 2.0 Example: COMPASS $\eta\pi$ scattering. $a_2\to \eta\pi$
We have been given a clean sample of the data for $\eta\pi$ scattering with the orbital momentum $L=2$:
$$
J^P:\qquad \underbrace{0^-}_{\eta} \otimes \underbrace{0^-}_{\pi} \otimes \underbrace{2^+}_{L=2} = 2^+
$$

In [ ]:
const tdata = let
    M = readdlm(joinpath("data", "EtaPi-2pp_arxiv1408.4286.txt"))
    Table(mηπ = M[:,1], I = M[:,2], δI = M[:,3])
end

In [ ]:
scatter(tdata.mηπ, tdata.I, yerr=tdata.δI, lc=:black, m=(2,:black))

In [ ]:
A(s; m, Γ) = m*Γ/(m^2-s-1im*m*Γ)

In [ ]:
scatter(tdata.mηπ, tdata.I, yerr=tdata.δI, lc=:black, m=(2,:black), lab="data")
plot!(mηπ->abs2(10*A(mηπ^2; m=1.1, Γ=0.4)), 1.0,2.0, l=:red, lab="simple model", title="hands adjustment")

***Exercise:*** adjust parameters $m$, $\Gamma$, and normalization

## 3. Advancing the model

In [ ]:
const mπ = 0.140; # GeV
const mη = 0.547; # GeV

In [ ]:
λ(x,y,z) = x^2+y^2+z^2-2x*y-2y*z-2z*x
function model(mηπ; pars)
    @unpack N0, m, Γ = pars
    #
    psq(x) = λ(x^2,mη^2,mπ^2)/(4x^2)
    bwf(z) = 1/(9+3z+z^2)
    FF(x;R = 3) = bwf(R^2*psq(x))/bwf(R^2*psq(m))
    #
    return abs2(N0*A(mηπ^2; m=m, Γ=Γ))*(psq(mηπ)/psq(m))^2*FF(mηπ)*sqrt(λ(mηπ^2,mη^2,mπ^2))/mηπ
end

In [ ]:
let pars = (N0 = 350, m=1.32,Γ=0.12)
    scatter(tdata.mηπ, tdata.I, yerr=tdata.δI, lc=:black, m=(2,:black), lab="data")
    plot!(mηπ->model(mηπ; pars=pars), 1.0,2.0, l=:red, lab="model", title="hands adjustment")
end

In [ ]:
loss(; data, pars) = sum((y - model(e; pars=pars))^2/δy^2 for (e, y, δy) in data)

In [ ]:
loss(; data=filter(x->1.0<x.mηπ<1.5, tdata), pars=(m=1.32,Γ=0.12,N0=350))

In [ ]:
fr = Optim.optimize(x->loss(; data=filter(x->1.0<x.mηπ<1.5, tdata), pars=(m=x[1],Γ=x[2],N0=x[3])),
    [1.32,0.12,350])

In [ ]:
let pars=NamedTuple{(:m,:Γ,:N0)}(Optim.minimizer(fr))
    scatter(tdata.mηπ, tdata.I, yerr=tdata.δI, lc=:black, m=(2,:black), label="data")
    plot!(mηπ->model(mηπ;pars=pars), 1.0,1.5, l=:red, label="model", title="fit")
end

In [ ]:
bestmodel(mηπ) = model(mηπ; pars=NamedTuple{(:m,:Γ,:N0)}(Optim.minimizer(fr)))
bestA(mηπ) = A(mηπ^2; NamedTuple{(:m,:Γ)}(Optim.minimizer(fr)[1:2])...)

In [ ]:
let
    plot()
    plot!(x->real(bestA(x)), 1.0,1.5, lab="Re")
    plot!(x->imag(bestA(x)), 1.0,1.5, lab="Im")
end

In [ ]:
let
    realmv = range(1.0,1.5, length=50)
    imagmv = range(-0.3,0, length=50)
    calv = [bestA(r+1im*i) for i in imagmv, r in realmv]
    calv = log10.(abs2.(calv))
    contour(realmv, imagmv, calv)
end

In [ ]:
pole_fr = Optim.optimize(x->abs2(1/bestA(x[1]+1im*x[2])), [1.3, -0.06])

In [ ]:
Optim.minimizer(pole_fr) .* [1, -2]

## $a_2'$ getting the higher-mass resonances

1. A general expression of the amplitude: numerator over the denominator (hehe)
$$
\begin{align}
A = \frac{N(s,\dots)}{D(s)}
\end{align}
$$
The numerator is production dependent, the denominator contains the resonance poles.

The general scattering theory controls only imaginary part of the denominator!
In the $\eta\pi$ example, Im$D=\text{const}$, does not change with $s$:
$$
\begin{align}
A^{(\text{simple})} = \frac{1}{(m^2-s)/(m\Gamma) - i}
\end{align}
$$

2. Incorporating the "unknown physics" in the parametrization of the numerator and the **demonimator**
 - once should do both, leaving as little freedom as possible in $D$,
 while $N$ must be a smouth function. No poles
$$
\begin{align}
A^{(\text{adv.})} = \frac{N(s)}{R(s) - i}
\end{align}
$$
Let's put a polynomical in $s$ in the $R(s)$.

In [ ]:
A2(s; ps) = 1/(sum(p*s^(i-1) for (i,p) in enumerate(ps))-1im)
function model2(mηπ; pars)
    @unpack N0, ps = pars
    #
    psq(x) = λ(x^2,mη^2,mπ^2)/(4x^2)
    bwf(z) = 1/(9+3z+z^2)
    FF(x;R = 3) = bwf(R^2*psq(x))
    #
    return abs2(N0*A2(mηπ^2; ps=ps))*(psq(mηπ))^2*FF(mηπ)*sqrt(λ(mηπ^2,mη^2,mπ^2))/mηπ
end

In [ ]:
let 
    m, Γ, N0 = Optim.minimizer(fr)
    pars=(ps = [m^2/(m*Γ), -1/(m*Γ), 0], N0=17*N0)
    scatter(tdata.mηπ, tdata.I, yerr=tdata.δI, lc=:black, m=(2,:black), label="data")
    plot!(mηπ->model2(mηπ;pars=pars), 1.0,1.5, l=:red, label="model", title="fit")
end

In [ ]:
function loss2(; data, pars)
    return sum((y - model2(e; pars=pars))^2/δy^2 for (e, y, δy) in data)
end

In [ ]:
fr2 = Optim.optimize(x->loss2(; data=filter(x->1.0<x.mηπ<2.0, tdata), pars=(N0=x[1],ps=x[2:end])),
#     [350,1.32/0.12,-1/(1.32*0.12),rand(),rand()])
    [350,rand(4)...])

In [ ]:
function makeanalysis(settings)
    data = settings["data"]
    Nps = settings["Nps"]
    Natt = settings["Natt"]
    frs = [Optim.optimize(x->loss2(; data=data, pars=(N0=x[1],ps=x[2:end])),
        [350,rand(Nps)...]) for e in 1:Natt]
    settings["fit_resultls"] = frs
    return settings
end

In [ ]:
settings = Dict(
    "data"=>filter(x->1.0<x.mηπ<2.0, tdata),
    "Nps"=>6,
    "Natt"=>500)
makeanalysis(settings);

In [ ]:
bestmin = findmin(Optim.minimum.(settings["fit_resultls"]))

In [ ]:
bestpars = Optim.minimizer(settings["fit_resultls"][bestmin[2]])
bestmodel2(mηπ) = model2(mηπ; pars=(N0=bestpars[1], ps=bestpars[2:end]))
bestA2(mηπ) = A2(mηπ^2; ps=bestpars[2:end])

In [ ]:
let 
    pars=(N0=bestpars[1], ps=bestpars[2:end])
    scatter(tdata.mηπ, tdata.I, yerr=tdata.δI, lc=:black, m=(2,:black), label="data")
    xmin, xmax = settings["data"].mηπ[[1,end]]
    plot!(bestmodel2, xmin, xmax, l=:red, label="model", title="fit")
end

In [ ]:
let
    realmv = range(1.0,2.2, length=50)
    imagmv = range(-0.5,0, length=50)
    calv = [bestA2(r+1im*i) for i in imagmv, r in realmv]
    calv = log10.(abs2.(calv))
    contour(realmv, imagmv, calv)
end

In [ ]:
a2′pole_fr = Optim.optimize(x->abs2(1/bestA2(x[1]+1im*x[2])), [1.9, -0.2])

In [ ]:
Optim.minimizer(a2′pole_fr) .* [1, -2]

See mode details in the research paper [Tensow wave by JPAC and COMPASS](https://inspirehep.net/literature/1665092).